In [7]:
#Import các thư viện
import pandas as pd
import numpy as np

# TASK 1: Mở các tập tin văn bản bên ngoài được yêu cầu với exception-handling
def open_file():
    while True:
        filename = input("Enter a class to grade (i.e. class1 for class1.txt): ") + ".txt"
        try:
            df = pd.read_csv(filename, sep='\t', header=None, names=['line'], dtype=str, engine='python')
            print(f"Successfully opened {filename}")
            return filename, df
        except FileNotFoundError:
            print("File cannot be found.")

filename, df_lines = open_file()

# TASK 2: Quét từng dòng của câu trả lời bài thi để tìm dữ liệu hợp lệ và cung cấp báo cáo tương ứng
def process_data(df_lines):
    print("\n**** ANALYZING ****")
    valid_data = []
    invalid_count = 0

    for raw_line in df_lines['line']:
        line = str(raw_line).strip()
        parts = line.split(',')

        # Kiểm tra số lượng giá trị
        if len(parts) != 26:
            print(f"Invalid line of data: does not contain exactly 26 values:\n{line}")
            invalid_count += 1
            continue

        # Kiểm tra ID học sinh: bắt đầu bằng 'N' + 8 chữ số
        student_id = parts[0]
        if not (student_id.startswith('N') and len(student_id) == 9 and student_id[1:].isdigit()):
            print(f"Invalid line of data: N# is invalid\n{line}")
            invalid_count += 1
            continue

        valid_data.append(parts)

    if invalid_count == 0:
        print("No errors found!")

    print("\n**** REPORT ****")
    print(f"Total valid lines of data: {len(valid_data)}")
    print(f"Total invalid lines of data: {invalid_count}")

    return valid_data

valid_data = process_data(df_lines)
                         
# TASK 3: Chấm điểm từng bài thi dựa trên tiêu chí đánh giá (rubric) được cung cấp và báo cáo
def grade_and_analyze(valid_data):
    answer_key = np.array("B,A,D,D,C,B,D,A,C,C,D,B,A,B,A,C,B,D,A,C,A,A,B,D,D".split(','))
    df_valid = pd.DataFrame(valid_data, columns=['ID'] + [f'Q{i+1}' for i in range(25)])

    # Tính điểm cho từng học sinh
    answers = df_valid.iloc[:, 1:].to_numpy()
    scores = np.where(answers == '', 0, np.where(answers == answer_key, 4, -1)).sum(axis=1)
    df_valid['Score'] = scores

    # Thống kê điểm
    print("\n**** GRADE REPORT ****")
    print(f"Total student of high scores: {(scores > 80).sum()}")
    print(f"Mean (average) score: {round(scores.mean(), 2)}")
    print(f"Highest score: {scores.max()}")
    print(f"Lowest score: {scores.min()}")
    print(f"Range of scores: {scores.max() - scores.min()}")
    median = np.median(scores)
    print(f"Median score: {int(median) if median % 1 == 0 else median}")

    
    # 3.7: Câu bị bỏ qua nhiều nhất
    skips = (df_valid.iloc[:, 1:26] == '').sum()
    total = len(df_valid)
    max_skip = skips.max()
    most_skipped = [(i+1, skips.iloc[i], round(skips.iloc[i]/total, 2)) for i in range(25) if skips.iloc[i] == max_skip]
    print("\nQuestion that most people skip:", ', '.join([f"{q} - {c} - {r}" for q, c, r in most_skipped]))
    
    # 3.8: Câu bị sai nhiều nhất
    wrongs = ((df_valid.iloc[:, 1:26] != answer_key) & (df_valid.iloc[:, 1:26] != '')).sum()
    max_wrong = wrongs.max()
    most_wrong = [(i+1, wrongs.iloc[i], round(wrongs.iloc[i]/total, 2)) for i in range(25) if wrongs.iloc[i] == max_wrong]
    print("Question that most people answer incorrectly:", ', '.join([f"{q} - {c} - {r:.2f}" for q, c, r in most_wrong]))
    
    return df_valid

# Gọi Task 3
df_valid = grade_and_analyze(valid_data)

# TASK 4: Tạo tập tin kết quả
def export_results(df_valid, filename):
    output_file = filename.replace('.txt', '_grades.txt')
    df_valid[['ID', 'Score']].to_csv(output_file, index=False, header=False)
    print(f"\nResults exported to: {output_file}")
    
export_results(df_valid, filename)

Enter a class to grade (i.e. class1 for class1.txt):  class1


Successfully opened class1.txt

**** ANALYZING ****
No errors found!

**** REPORT ****
Total valid lines of data: 20
Total invalid lines of data: 0

**** GRADE REPORT ****
Total student of high scores: 6
Mean (average) score: 75.6
Highest score: 91
Lowest score: 59
Range of scores: 32
Median score: 73

Question that most people skip: 3 - 4 - 0.2, 5 - 4 - 0.2, 23 - 4 - 0.2
Question that most people answer incorrectly: 10 - 4 - 0.20, 14 - 4 - 0.20, 16 - 4 - 0.20, 19 - 4 - 0.20, 22 - 4 - 0.20

Results exported to: class1_grades.txt
